<a href="https://colab.research.google.com/github/mengyanl/springboard/blob/main/RDD_3rd_Google_Colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDD 3rd Google Colab notebook

In [2]:
!ls

sample_data


# Download and install Spark

In [3]:
# download java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Installing Spark 3.2.0 with Hadoop 2.7
!wget -q https://downloads.apache.org/spark/spark-3.2.0//spark-3.2.0-bin-hadoop2.7.tgz
# unzip the folder
!tar xf spark-3.2.0-bin-hadoop2.7.tgz
# install ‘findspark’ library. 
#It will locate Spark on the system and import it as a regular library.
!pip install -q findspark

# Setup environment

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

'/content/spark-3.2.0-bin-hadoop2.7'

In [10]:
!ls

sample_data  spark-3.2.0-bin-hadoop2.7	spark-3.2.0-bin-hadoop2.7.tgz


# Download Chicago Plolice Station Data

In [11]:
# get police station data
!wget -O plice_station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2021-11-24 04:34:07--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘plice_station.csv’

plice_station.csv       [ <=>                ]   5.57K  --.-KB/s    in 0s      

2021-11-24 04:34:08 (518 MB/s) - ‘plice_station.csv’ saved [5699]



# RDDs setup

In [12]:
!ls
ps = spark.read.csv('plice_station.csv', header = True)
ps.show(5)

plice_station.csv  spark-3.2.0-bin-hadoop2.7
sample_data	   spark-3.2.0-bin-hadoop2.7.tgz
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-

In [13]:
psrdd = sc.textFile('plice_station.csv')
psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [14]:
ps_header = psrdd.first()

# How many police stations are there?

In [15]:
ps_rest = psrdd.filter(lambda line: line!= ps_header)
ps_rest.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

In [16]:
ps_rest.map(lambda line: line.split(',')).collect()

[['Headquarters',
  'Headquarters',
  '3510 S Michigan Ave',
  'Chicago',
  'IL',
  '60653',
  'http://home.chicagopolice.org',
  '',
  '',
  '',
  '1177731.401',
  '1881697.404',
  '41.83070169',
  '-87.62339535',
  '"(41.8307016873',
  ' -87.6233953459)"'],
 ['18',
  'Near North',
  '1160 N Larrabee St',
  'Chicago',
  'IL',
  '60610',
  'http://home.chicagopolice.org/community/districts/18th-district-near-north/',
  '312-742-5870',
  '312-742-5771',
  '312-742-5773',
  '1172080.029',
  '1908086.527',
  '41.90324165',
  '-87.64335214',
  '"(41.9032416531',
  ' -87.6433521393)"'],
 ['19',
  'Town Hall',
  '850 W Addison St',
  'Chicago',
  'IL',
  '60613',
  'http://home.chicagopolice.org/community/districts/19th-district-town-hall/',
  '312-744-8320',
  '312-744-4481',
  '312-744-8011',
  '1169730.744',
  '1924160.317',
  '41.94740046',
  '-87.65151202',
  '"(41.9474004564',
  ' -87.651512018)"'],
 ['20',
  'Lincoln',
  '5400 N Lincoln Ave',
  'Chicago',
  'IL',
  '60625',
  'http://

In [19]:
ps_rest.map(lambda line: line.split(',')).count()

24

# Display the District ID, District name, Address and Zip for the police station with District ID 7

In [22]:
(ps_rest.filter(lambda line: line.split(',')[0] == '7').
   map(lambda line: (line.split(',')[0],
                     line.split(',')[1],
                     line.split(',')[2],
                     line.split(',')[5]
                     )).collect())

[('7', 'Englewood', '1438 W 63rd St', '60636')]

# Police stations 10 and 11 are geographically close to each other. Display the District ID, District name, address and zip code


In [23]:
(ps_rest.filter(lambda line: line.split(',')[0] in ['10', '11']).
    map(lambda line: (line.split(',')[1],
                      line.split(',')[2],
                      line.split(',')[5]
    )).collect())
        

[('Ogden', '3315 W Ogden Ave', '60623'),
 ('Harrison', '3151 W Harrison St', '60612')]